In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shusrith/machine-trainslation")

print("Path to dataset files:", path)

100%|██████████| 174M/174M [00:02<00:00, 62.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shusrith/machine-trainslation/versions/5


In [ ]:
!ls $path/Dataset1

Dataset1


In [ ]:
import pandas as pd

df = pd.read_csv(f"{path}/Dataset1/Dataset1/spa.txt.csv")
df

,English,Translated
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.
...,...,...
128079,There are four main causes of alcohol-related ...,Hay cuatro causas principales de muertes relac...
128080,There are mothers and fathers who will lie awa...,Hay madres y padres que se quedan despiertos d...
128081,A carbon footprint is the amount of carbon dio...,Una huella de carbono es la cantidad de contam...
128082,Since there are usually multiple websites on a...,Como suele haber varias páginas web sobre cual...


In [ ]:
import re

def preproc(x):
    x = re.sub(r"[^a-zA-Z0-9\s]", "", x)
    x = x.lower()
    return x

def preproc_spanish(text):
    text = re.sub(r"[^a-zA-Z0-9áéíóúüñÁÉÍÓÚÜÑ\s]", "", text)
    text = text.lower()
    return text

df["English"] = df["English"].apply(preproc)
df["Spanish"] = df["Translated"].apply(preproc_spanish)
df.drop(columns=["Translated"], inplace=True)
df

,English,Spanish
0,go,ve
1,go,vete
2,go,vaya
3,go,váyase
4,hi,hola
...,...,...
128079,there are four main causes of alcoholrelated d...,hay cuatro causas principales de muertes relac...
128080,there are mothers and fathers who will lie awa...,hay madres y padres que se quedan despiertos d...
128081,a carbon footprint is the amount of carbon dio...,una huella de carbono es la cantidad de contam...
128082,since there are usually multiple websites on a...,como suele haber varias páginas web sobre cual...


In [ ]:
corpus = " ".join(df["English"].tolist())
with open("corpus.txt", "w") as f:
    f.write(corpus)

In [ ]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(["corpus.txt"], vocab_size=30000)

In [ ]:
eng_vocab = tokenizer.get_vocab()
eng_vocab = sorted(eng_vocab.items(), key=lambda x: x[1])
eng_vocab

[('!', 0),
 ('"', 1),
 ('#', 2),
 ('$', 3),
 ('%', 4),
 ('&', 5),
 ("'", 6),
 ('(', 7),
 (')', 8),
 ('*', 9),
 ('+', 10),
 (',', 11),
 ('-', 12),
 ('.', 13),
 ('/', 14),
 ('0', 15),
 ('1', 16),
 ('2', 17),
 ('3', 18),
 ('4', 19),
 ('5', 20),
 ('6', 21),
 ('7', 22),
 ('8', 23),
 ('9', 24),
 (':', 25),
 (';', 26),
 ('<', 27),
 ('=', 28),
 ('>', 29),
 ('?', 30),
 ('@', 31),
 ('A', 32),
 ('B', 33),
 ('C', 34),
 ('D', 35),
 ('E', 36),
 ('F', 37),
 ('G', 38),
 ('H', 39),
 ('I', 40),
 ('J', 41),
 ('K', 42),
 ('L', 43),
 ('M', 44),
 ('N', 45),
 ('O', 46),
 ('P', 47),
 ('Q', 48),
 ('R', 49),
 ('S', 50),
 ('T', 51),
 ('U', 52),
 ('V', 53),
 ('W', 54),
 ('X', 55),
 ('Y', 56),
 ('Z', 57),
 ('[', 58),
 ('\\', 59),
 (']', 60),
 ('^', 61),
 ('_', 62),
 ('`', 63),
 ('a', 64),
 ('b', 65),
 ('c', 66),
 ('d', 67),
 ('e', 68),
 ('f', 69),
 ('g', 70),
 ('h', 71),
 ('i', 72),
 ('j', 73),
 ('k', 74),
 ('l', 75),
 ('m', 76),
 ('n', 77),
 ('o', 78),
 ('p', 79),
 ('q', 80),
 ('r', 81),
 ('s', 82),
 ('t', 83),
 

In [ ]:
corpus = " ".join(df["Spanish"].tolist())
with open("corpus.txt", "w") as f:
    f.write(corpus)

In [ ]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(["corpus.txt"], vocab_size=30000)

In [ ]:
spa_vocab = tokenizer.get_vocab()
spa_vocab = sorted(spa_vocab.items(), key=lambda x: x[1])
spa_vocab

[('!', 0),
 ('"', 1),
 ('#', 2),
 ('$', 3),
 ('%', 4),
 ('&', 5),
 ("'", 6),
 ('(', 7),
 (')', 8),
 ('*', 9),
 ('+', 10),
 (',', 11),
 ('-', 12),
 ('.', 13),
 ('/', 14),
 ('0', 15),
 ('1', 16),
 ('2', 17),
 ('3', 18),
 ('4', 19),
 ('5', 20),
 ('6', 21),
 ('7', 22),
 ('8', 23),
 ('9', 24),
 (':', 25),
 (';', 26),
 ('<', 27),
 ('=', 28),
 ('>', 29),
 ('?', 30),
 ('@', 31),
 ('A', 32),
 ('B', 33),
 ('C', 34),
 ('D', 35),
 ('E', 36),
 ('F', 37),
 ('G', 38),
 ('H', 39),
 ('I', 40),
 ('J', 41),
 ('K', 42),
 ('L', 43),
 ('M', 44),
 ('N', 45),
 ('O', 46),
 ('P', 47),
 ('Q', 48),
 ('R', 49),
 ('S', 50),
 ('T', 51),
 ('U', 52),
 ('V', 53),
 ('W', 54),
 ('X', 55),
 ('Y', 56),
 ('Z', 57),
 ('[', 58),
 ('\\', 59),
 (']', 60),
 ('^', 61),
 ('_', 62),
 ('`', 63),
 ('a', 64),
 ('b', 65),
 ('c', 66),
 ('d', 67),
 ('e', 68),
 ('f', 69),
 ('g', 70),
 ('h', 71),
 ('i', 72),
 ('j', 73),
 ('k', 74),
 ('l', 75),
 ('m', 76),
 ('n', 77),
 ('o', 78),
 ('p', 79),
 ('q', 80),
 ('r', 81),
 ('s', 82),
 ('t', 83),
 

In [ ]:
e_vocab = {i : j for i, j in eng_vocab}
s_vocab = {i : j for i, j in spa_vocab}
e_vocab_len = len(e_vocab)
s_vocab_len = len(s_vocab)
e_vocab["<PAD>"] = e_vocab_len + 4
e_vocab["<UNK>"] = e_vocab_len + 1
e_vocab["<SOS>"] = e_vocab_len + 2
e_vocab["<EOS>"] = e_vocab_len + 3
s_vocab["<PAD>"] = s_vocab_len + 4
s_vocab["<UNK>"] = s_vocab_len + 1
s_vocab["<SOS>"] = s_vocab_len + 2
s_vocab["<EOS>"] = s_vocab_len + 3

In [ ]:
import json
with open("eng_vocab_bpe.json", "w", encoding="utf-8") as f:
    json.dump(e_vocab, f, ensure_ascii=False, indent=4)
    f.close()

import json

with open("spa_vocab_bpe.json", "w", encoding="utf-8") as f:
    json.dump(s_vocab, f, ensure_ascii=False, indent=4)

In [ ]:
def encode(text, vocab):
    return [vocab.get(word, vocab["<UNK>"]) for word in text.split()]

df["English"] = df["English"].apply(lambda x: encode(x, e_vocab))
df["Spanish"] = df["Spanish"].apply(lambda x: encode(x, s_vocab))
df

,English,Spanish
0,[4024],[491]
1,[4024],[10639]
2,[4024],[23720]
3,[4024],[23720]
4,[7954],[23720]
...,...,...
128079,"[14450, 714, 7053, 14450, 14450, 11907, 14450,...","[23720, 23720, 23720, 23720, 331, 23720, 23720..."
128080,"[14450, 714, 14450, 462, 14450, 14450, 14450, ...","[23720, 23720, 88, 23720, 568, 424, 23720, 237..."
128081,"[64, 14450, 14450, 283, 14450, 14450, 11907, 1...","[8327, 23720, 331, 23720, 277, 312, 23720, 331..."
128082,"[14450, 14450, 714, 14450, 14450, 14450, 277, ...","[23720, 23720, 23720, 23720, 23720, 23720, 237..."


In [ ]:
length_counts = df["English"].apply(len).value_counts().sort_index(ascending=False)
cumulative_counts = length_counts.cumsum()[::-1]
cumulative_counts

,count
English,
1,128084
2,128018
3,124761
4,114016
5,94735
6,72168
7,50863
8,33431
9,21332


In [ ]:
length_counts = df["Spanish"].apply(len).value_counts().sort_index(ascending=False)
cumulative_counts = length_counts.cumsum()[::-1]
cumulative_counts

,count
Spanish,
1,128084
2,127324
3,122001
4,108153
5,87430
6,65786
7,46446
8,31089
9,20138


In [ ]:
import pandas as pd

max_len = 13
new_data = []

for _, row in df.iterrows():
    english = row["English"]
    english.append(e_vocab["<EOS>"])
    spanish = row["Spanish"]
    spanish.insert(0, s_vocab["<SOS>"])
    spanish.append(s_vocab["<EOS>"])

    eng_chunks = []
    spa_chunks = []

    for i in range(0, len(english), max_len - 1):
        eng_chunk = english[i : i + max_len - 1]
        while len(eng_chunk) < max_len:
            eng_chunk.append(e_vocab["<PAD>"])
        eng_chunks.append(eng_chunk)

    for i in range(0, len(spanish), max_len - 1):
        spa_chunk = spanish[i : i + max_len - 1]
        while len(spa_chunk) < max_len:
            spa_chunk.append(s_vocab["<PAD>"])
        spa_chunks.append(spa_chunk)

    chunks = list(zip(eng_chunks, spa_chunks))
    new_data.extend(chunks)

df1 = pd.DataFrame(new_data, columns=["English", "Spanish"])
df1

,English,Spanish
0,"[4024, 14452, 14453, 14453, 14453, 14453, 1445...","[23721, 491, 23722, 23723, 23723, 23723, 23723..."
1,"[4024, 14452, 14453, 14453, 14453, 14453, 1445...","[23721, 10639, 23722, 23723, 23723, 23723, 237..."
2,"[4024, 14452, 14453, 14453, 14453, 14453, 1445...","[23721, 23720, 23722, 23723, 23723, 23723, 237..."
3,"[4024, 14452, 14453, 14453, 14453, 14453, 1445...","[23721, 23720, 23722, 23723, 23723, 23723, 237..."
4,"[7954, 14452, 14453, 14453, 14453, 14453, 1445...","[23721, 23720, 23722, 23723, 23723, 23723, 237..."
...,...,...
132047,"[14450, 462, 14450, 8559, 14450, 1384, 14450, ...","[23720, 932, 23720, 88, 23720, 23720, 23720, 2..."
132048,"[539, 11964, 14450, 5507, 14450, 9330, 64, 103...","[23721, 2719, 23720, 23720, 23720, 281, 23720,..."
132049,"[14450, 5507, 14450, 14450, 14450, 14450, 1445...","[23720, 23720, 312, 23720, 23720, 8327, 88, 23..."
132050,"[14450, 1972, 14450, 14450, 14450, 14450, 1445...","[23720, 268, 568, 281, 23720, 331, 23720, 2372..."


In [ ]:
df2 = df.copy()
def pad(x, max_len, vocab):
    while len(x) < max_len:
        x.append(vocab["<PAD>"])
    return x
df2["English"] = df2["English"].apply(lambda x: pad(x, 48, e_vocab))
df2["Spanish"] = df2["Spanish"].apply(lambda x: pad(x, 51, s_vocab))
df2

,English,Spanish
0,"[4024, 14452, 14453, 14453, 14453, 14453, 1445...","[23721, 491, 23722, 23723, 23723, 23723, 23723..."
1,"[4024, 14452, 14453, 14453, 14453, 14453, 1445...","[23721, 10639, 23722, 23723, 23723, 23723, 237..."
2,"[4024, 14452, 14453, 14453, 14453, 14453, 1445...","[23721, 23720, 23722, 23723, 23723, 23723, 237..."
3,"[4024, 14452, 14453, 14453, 14453, 14453, 1445...","[23721, 23720, 23722, 23723, 23723, 23723, 237..."
4,"[7954, 14452, 14453, 14453, 14453, 14453, 1445...","[23721, 23720, 23722, 23723, 23723, 23723, 237..."
...,...,...
128079,"[14450, 714, 7053, 14450, 14450, 11907, 14450,...","[23721, 23720, 23720, 23720, 23720, 331, 23720..."
128080,"[14450, 714, 14450, 462, 14450, 14450, 14450, ...","[23721, 23720, 23720, 88, 23720, 568, 424, 237..."
128081,"[64, 14450, 14450, 283, 14450, 14450, 11907, 1...","[23721, 8327, 23720, 331, 23720, 277, 312, 237..."
128082,"[14450, 14450, 714, 14450, 14450, 14450, 277, ...","[23721, 23720, 23720, 23720, 23720, 23720, 237..."


In [ ]:
df.to_csv("output_bpe.csv", index=False)
df1.to_csv("output1_bpe.csv", index=False)
df2.to_csv("output2_bpe.csv", index=False)

In [ ]:
length_counts = df2["Spanish"].apply(len).value_counts().sort_index(ascending=False)
cumulative_counts = length_counts.cumsum()[::-1]
cumulative_counts

,count
Spanish,
51,128084
